# Output Parsers

Os analisadores de saída são responsáveis por pegar a saída de um LLM e transformá-la em um formato mais adequado. Isso é muito útil quando você está usando LLMs para gerar qualquer forma de dados estruturados.

Digamos que temos a seguinte review de um produto:

> "Este soprador de folhas é bastante incrível. Ele tem quatro configurações: sopro de vela, brisa suave, cidade ventosa e tornado. Chegou em dois dias, bem a tempo para o presente de aniversário da minha esposa. Acho que minha esposa gostou tanto que ficou sem palavras. Até agora, fui o único a usá-lo, e tenho usado em todas as manhãs alternadas para limpar as folhas do nosso gramado. É um pouco mais caro do que os outros sopradores de folhas disponíveis no mercado, mas acho que vale a pena pelas características extras."

E eu quero que o modelo de linguagem processe esta review para estruturá-la no seguinte formato:

```json
{
  "presente": true,
  "dias_entrega": 2,
  "percepcao_de_valor": ["um pouco mais caro do que os outros sopradores de folhas disponíveis no mercado"]
}

```



In [11]:
review_cliente = """Este soprador de folhas é bastante incrível. Ele tem 
quatro configurações: sopro de vela, brisa suave, cidade ventosa 
e tornado. Chegou em dois dias, bem a tempo para o presente de 
aniversário da minha esposa. Acho que minha esposa gostou tanto 
que ficou sem palavras. Até agora, fui o único a usá-lo, e tenho 
usado em todas as manhãs alternadas para limpar as folhas do 
nosso gramado. É um pouco mais caro do que os outros sopradores 
de folhas disponíveis no mercado, mas acho que vale a pena pelas 
características extras."""


from langchain.prompts import ChatPromptTemplate

review_template = ChatPromptTemplate.from_template("""
Para o texto a seguir, extraia as seguintes informações:

presente: O item foi comprado para algém? True caso verdadeiro
e False caso falos ou não tenha a infomação.

dias_entrega: Quantos dias para a entrega chegar?
Se a resposta não for encontrada, retorne -1.

percepcao_de_valor: Extraia qualquer frase sobre o 
valor ou preço do produto. Retorne como uma lista
de Python.

Texto: {review}

Retorn a respostra no formato JSON
""")

print(review_template.format_messages(review=review_cliente))

[HumanMessage(content='\nPara o texto a seguir, extraia as seguintes informações:\n\npresente: O item foi comprado para algém? True caso verdadeiro\ne False caso falos ou não tenha a infomação.\n\ndias_entrega: Quantos dias para a entrega chegar?\nSe a resposta não for encontrada, retorne -1.\n\npercepcao_de_valor: Extraia qualquer frase sobre o \nvalor ou preço do produto. Retorne como uma lista\nde Python.\n\nTexto: Este soprador de folhas é bastante incrível. Ele tem \nquatro configurações: sopro de vela, brisa suave, cidade ventosa \ne tornado. Chegou em dois dias, bem a tempo para o presente de \naniversário da minha esposa. Acho que minha esposa gostou tanto \nque ficou sem palavras. Até agora, fui o único a usá-lo, e tenho \nusado em todas as manhãs alternadas para limpar as folhas do \nnosso gramado. É um pouco mais caro do que os outros sopradores \nde folhas disponíveis no mercado, mas acho que vale a pena pelas \ncaracterísticas extras.\n\nRetorn a respostra no formato JSON\

In [12]:
from langchain_openai.chat_models import ChatOpenAI

chat = ChatOpenAI()
resposta = chat.invoke(review_template.format_messages(review=review_cliente))

In [1]:
# resposta.content['presente'] -> erro precisa parsear

In [5]:
print(resposta.content)

{
    "presente": true,
    "dias_entrega": 2,
    "percepcao_de_valor": ["É um pouco mais caro do que os outros sopradores de folhas disponíveis no mercado, mas acho que vale a pena pelas características extras."]
}


In [6]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

In [7]:
schema_presente = ResponseSchema(
    name='presente',
    type='bool',
    description='O item foi comprado para algém? True caso verdadeiro \
e False caso falos ou não tenha a infomação.'
)

schema_entrega = ResponseSchema(
    name='dias_entrega',
    type='int',
    description='Quantos dias para a entrega chegar?\
Se a resposta não for encontrada, retorne -1.'
)

schema_valor = ResponseSchema(
    name='percepcao_de_valor',
    type='list',
    description='Extraia qualquer frase sobre o \
valor ou preço do produto. Retorne como uma lista \
de Python.'
)

response_schema = [schema_presente,
                   schema_entrega,
                   schema_valor]
output_parser = StructuredOutputParser.from_response_schemas(response_schema)
schema_formatado = output_parser.get_format_instructions()
print(schema_formatado)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"presente": bool  // O item foi comprado para algém? True caso verdadeiro e False caso falos ou não tenha a infomação.
	"dias_entrega": int  // Quantos dias para a entrega chegar?Se a resposta não for encontrada, retorne -1.
	"percepcao_de_valor": list  // Extraia qualquer frase sobre o valor ou preço do produto. Retorne como uma lista de Python.
}
```


In [16]:

from langchain.prompts import ChatPromptTemplate

review_template = ChatPromptTemplate.from_template("""
Para o texto a seguir, extraia as seguintes informações:

presente, dias_entrega e percepcao_de_valor

Texto: {review}

{schema}
""", partial_variables={'schema': schema_formatado})

print(review_template.format_messages(review=review_cliente))

[HumanMessage(content='\nPara o texto a seguir, extraia as seguintes informações:\n\npresente, dias_entrega e percepcao_de_valor\n\nTexto: Este soprador de folhas é bastante incrível. Ele tem \nquatro configurações: sopro de vela, brisa suave, cidade ventosa \ne tornado. Chegou em dois dias, bem a tempo para o presente de \naniversário da minha esposa. Acho que minha esposa gostou tanto \nque ficou sem palavras. Até agora, fui o único a usá-lo, e tenho \nusado em todas as manhãs alternadas para limpar as folhas do \nnosso gramado. É um pouco mais caro do que os outros sopradores \nde folhas disponíveis no mercado, mas acho que vale a pena pelas \ncaracterísticas extras.\n\nThe output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":\n\n```json\n{\n\t"presente": bool  // O item foi comprado para algém? True caso verdadeiro e False caso falos ou não tenha a infomação.\n\t"dias_entrega": int  // Quantos dias para a

In [17]:
from langchain_openai.chat_models import ChatOpenAI

chat = ChatOpenAI()
resposta = chat.invoke(review_template.format_messages(review=review_cliente))
resposta

AIMessage(content='```json\n{\n\t"presente": true,\n\t"dias_entrega": 2,\n\t"percepcao_de_valor": ["É um pouco mais caro do que os outros sopradores de folhas disponíveis no mercado, mas acho que vale a pena pelas características extras."]\n}\n```', response_metadata={'token_usage': {'completion_tokens': 63, 'prompt_tokens': 320, 'total_tokens': 383}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_c2295e73ad', 'finish_reason': 'stop', 'logprobs': None}, id='run-db4d9066-b4f7-4b82-a318-82e9957a9c89-0')

In [18]:
print(resposta.content)

```json
{
	"presente": true,
	"dias_entrega": 2,
	"percepcao_de_valor": ["É um pouco mais caro do que os outros sopradores de folhas disponíveis no mercado, mas acho que vale a pena pelas características extras."]
}
```


In [19]:
resposta_json = output_parser.parse(resposta.content)

In [21]:
resposta_json['presente']

True

In [22]:
resposta_json['dias_entrega']

2

In [23]:
resposta_json['percepcao_de_valor']

['É um pouco mais caro do que os outros sopradores de folhas disponíveis no mercado, mas acho que vale a pena pelas características extras.']